<a href="https://colab.research.google.com/github/umleeho1/DataAnalysis/blob/main/Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import folium

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Colab Notebooks/DataType/maps/seoul-metro-2021.logs.csv'
data = pd.read_csv(path)

data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,timestamp,station_code,people_in,people_out
0,2021-01-01T05:00:00.000+09:00,150,86,85
1,2021-01-01T06:00:00.000+09:00,150,111,355
2,2021-01-01T07:00:00.000+09:00,150,157,438
3,2021-01-01T08:00:00.000+09:00,150,306,592
4,2021-01-01T09:00:00.000+09:00,150,333,841
...,...,...,...,...
1941548,2021-12-31T19:00:00.000+09:00,2828,74,263
1941549,2021-12-31T20:00:00.000+09:00,2828,73,145
1941550,2021-12-31T21:00:00.000+09:00,2828,95,209
1941551,2021-12-31T22:00:00.000+09:00,2828,54,138


In [5]:
station_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataType/maps/seoul-metro-station-info.csv')
station_info = station_info[['station.code',
                              'geo.latitude',
                              'geo.longitude']
                             ].set_index('station.code')
station_info

,geo.latitude,geo.longitude
station.code,,
158,37.580178,127.046835
157,37.578103,127.034893
156,37.575297,127.025087
159,37.572627,127.016429
155,37.571420,127.009745
...,...,...
2823,37.451535,127.159816
2824,37.445210,127.156866
2825,37.440918,127.147564


In [6]:
seoul = folium.Map(location=[37.55, 126.98], zoom_start = 12)
seoul

In [7]:
lunch_data = data[(pd.to_datetime(data.timestamp).dt.hour>10)
                    & (pd.to_datetime(data.timestamp).dt.hour<15)]
lunch_data

,timestamp,station_code,people_in,people_out
6,2021-01-01T11:00:00.000+09:00,150,579,625
7,2021-01-01T12:00:00.000+09:00,150,737,838
8,2021-01-01T13:00:00.000+09:00,150,696,788
9,2021-01-01T14:00:00.000+09:00,150,674,791
25,2021-01-01T11:00:00.000+09:00,151,117,208
...,...,...,...,...
1941524,2021-12-31T14:00:00.000+09:00,2827,325,186
1941540,2021-12-31T11:00:00.000+09:00,2828,240,112
1941541,2021-12-31T12:00:00.000+09:00,2828,192,124
1941542,2021-12-31T13:00:00.000+09:00,2828,189,159


In [8]:
lunch_station_sum = lunch_data.groupby("station_code").sum()
lunch_station_sum

<ipython-input-8-daf76a630668>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  lunch_station_sum = lunch_data.groupby("station_code").sum()


,people_in,people_out
station_code,,
150,2505050,2386392
151,824481,1122552
152,1414527,1829519
153,1848245,2472974
154,1652295,2041594
...,...,...
2824,604018,448206
2825,298017,277076
2826,339413,271084


In [9]:
lunch_joined_data = lunch_station_sum.join(station_info)

In [10]:
from folium.plugins import HeatMap

#점심 시간의 승차 인원
lunch_seoul_in = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data = lunch_joined_data[['geo.latitude',
                                  'geo.longitude',
                                  'people_in']
                                ]).add_to(lunch_seoul_in)
lunch_seoul_in

In [11]:
#점심 시간의 하차 인원
lunch_seoul_out = folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data = lunch_joined_data[['geo.latitude',
                                  'geo.longitude',
                                  'people_out']
                                ]).add_to(lunch_seoul_out)
lunch_seoul_out

In [12]:
['geo.latitude', 'geo.longitute', 'people_out']

['geo.latitude', 'geo.longitute', 'people_out']

In [13]:
lunch_seoul_out_circle = folium.Map(location=[37.55, 126.98], zoom_start = 12)

for index, row in lunch_joined_data[['geo.latitude',
                                  'geo.longitude',
                                  'people_out']
                                ].iterrows():
  if int(row['people_out']) > 2_500_000:
    folium.Circle(
        location=[str(row['geo.latitude']),
                  str(row['geo.longitude'])],
        color='Red',
        fill=True,
        radius=(int(row['people_out'])/10000)
    ).add_to(lunch_seoul_out_circle)

lunch_seoul_out_circle